In [1]:
!ls

combined  data1.tar.bz2  NYdata  titanlib_test.ipynb


In [13]:
import os
import numpy as np
import pandas as pd



df=pd.DataFrame();
def Create_df(name):
    
    
    df= pd.DataFrame(np.loadtxt(name), columns=['station','timestamp','min', 'max'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m%d%H%M')
    #df.describe()
    #df['station'].unique()
    df['file_type']= dirname

    #print(df)

    return df

for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        if 'Press' in filename:
            #print(dirname+filename)
            df2=Create_df(str(dirname+'/'+filename))
            df=df.append(df2, ignore_index=True)

        
df['min']=df['min'].replace(0, np.nan)        
df['max']=df['max'].replace(0, np.nan) 

print(df)



        station           timestamp     min      max      file_type
0        4201.0 1997-06-10 00:00:00  1008.7  1012.45  data/combined
1        4201.0 1997-06-10 01:00:00  1009.0  1012.76  data/combined
2        4201.0 1997-06-10 02:00:00  1009.2  1012.96  data/combined
3        4201.0 1997-06-10 03:00:00  1009.2  1012.98  data/combined
4        4201.0 1997-06-10 04:00:00  1009.5  1013.28  data/combined
...         ...                 ...     ...      ...            ...
197599   6193.0 1997-06-28 09:00:00  1007.3  1008.60    data/NYdata
197600   6193.0 1997-06-28 12:00:00  1008.3  1009.60    data/NYdata
197601   6193.0 1997-06-28 15:00:00  1010.1  1011.40    data/NYdata
197602   6193.0 1997-06-28 18:00:00  1011.4  1012.70    data/NYdata
197603   6193.0 1997-06-28 21:00:00  1012.6  1013.90    data/NYdata

[197604 rows x 5 columns]


In [14]:
df_true= df[df['file_type']=='data/Carra']
df=df[df['file_type']=='data/combined']
print(df['station'].unique())

[4201. 4202. 4203. 4207. 4208. 4210. 4214. 4217. 4220. 4221. 4228. 4230.
 4231. 4242. 4250. 4253. 4260. 4261. 4266. 4270. 4272. 4285. 4301. 4310.
 4312. 4313. 4320. 4330. 4339. 4351. 4360. 4382. 4390. 5700. 6005. 6009.
 6010. 6011. 6024. 6030. 6034. 6041. 6043. 6048. 6052. 6058. 6060. 6070.
 6071. 6075. 6079. 6080. 6081. 6096. 6100. 6104. 6108. 6110. 6119. 6120.
 6124. 6142. 6143. 6149. 6151. 6153. 6159. 6160. 6165. 6168. 6169. 6170.
 6180. 6181. 6184. 6190. 6191. 6193.]


In [21]:
from scipy.interpolate import interp1d
def create_sets(station, df=df):
    """
    Parameters
    ----------
        - station: Station ID used for computation
        - df: dataframe from which the data has to be queried
        
    Outputs
    -------
        - x: Time stamps of the station 
        - f: Cubic spline closure to get the y value by interpolation at any given time.  
    """
    df2=df[df['station']==station]

    x=df2['timestamp']
    y=df2['max']

    x = x.to_numpy('float64')
    y = y.to_numpy('float64')

    X = np.array([x,y])
    X= X.T
    X=X[X[:, 0].argsort()]
    x= X[:,0]
    y= X[:,1]

    f= interp1d(x, y, kind='cubic', fill_value='extrapolate' )
    
    return x,f

x,f= create_sets(6096)

print(x)

[8.624448e+17 8.624484e+17 8.624520e+17 ... 8.677044e+17 8.677080e+17
 8.677116e+17]


In [77]:
def divide(x):
    return int(x/100)

df3= pd.DataFrame(np.loadtxt('data/SynopFrIngres'),columns=['station', 'init', 'end', 'lat', 'lon', 'height'])

df3['station']= df3['station'].apply(divide)
df3= df3.drop_duplicates(subset='station')
#positions= df3[['lat', 'lon']].values

# TODO: remove drop duplicates and find a solution for updating the position of the stations dynamically


def get_stations(station, radius):
        
    lon= df3[df3['station']==station]['lon'].to_numpy()[0]
    lat= df3[df3['station']==station]['lat'].to_numpy()[0]    
    stations_in_range = df3[df3['lat'] > lat-radius ][df3['lat'] < lat+radius][df3['lon'] < lon+radius][df3['lon'] > lon-radius]
    return stations_in_range
    
    
print(get_stations(6096, 1)
      
def prepare_test(station):
      surrounds=get_stations(6096, 1)
      for sation in surrounds['station']
#TODO tomorrow: get stations in a certain range and perform the test. Wrap up into a function that focuses on one single station 



     station        init         end      lat     lon  height
10      5330  19860401.0  20021231.0  56.0636  9.1490    58.0
11      5430  19860401.0  19990430.0  55.6980  8.5555    23.0
294     5192  19790101.0  20131116.0  56.1603  9.5589    26.0
297     5195  20090825.0  20991231.0  56.1010  9.5530    92.0
298     5199  20130917.0  20991231.0  56.0761  9.3298    85.0
..       ...         ...         ...      ...     ...     ...
771     6109  19860100.0  20020424.0  55.4715  9.1124    62.0
773     6110  19530101.0  19850116.0  55.2252  9.2641    41.0
776     6113  19810901.0  20010101.0  55.1796  9.0711    48.0
777     6115  19610101.0  19741201.0  54.9073  8.8331     1.0
779     6116  19840905.0  19880624.0  54.8987  9.1230    15.0

[75 rows x 6 columns]


/tmp/ipykernel_33389/499121237.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  stations_in_range = df3[df3['lat'] > lat-radius ][df3['lat'] < lat+radius][df3['lon'] < lon+radius][df3['lon'] > lon-radius]


In [6]:
threshold = 2
max_elev_diff = 200
elev_gradient = -0.0065
min_std = 1
num_iterations = 5

flags = titanlib.buddy_check(points, temp_obs, radius, num_min,threshold, max_elev_diff, elev_gradient, min_std, num_iterations)



NameError: name 'points' is not defined